# 第 9 章 實戰演練

In [1]:
import os
import pandas as pd

from openai import OpenAI
from pytube import YouTube
from rich import print as pprint

from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import YouTubeSearchTool
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader

In [2]:
os.environ['OPENAI_API_KEY'] = "sk-None-vowLahS2p4mOq6FP56VCT3BlbkFJTY1umKuhsfu61iHTNVDc"

## 9-1 Youtube 懶人包問答機器人
本節中將建立一個將YouTube上的影評的文字轉為RAG的方式來建立問答機器人

### 1. 建立 Youtube 查詢工具
使用`YouTubeSearchTool`建立查詢工具並處理格式給`pytube`模組使用

In [3]:
def YouTube_Search_Tool(keyword) -> str:
    keyword: str
    tool = YouTubeSearchTool()
    result = tool.invoke(keyword)
    urls = eval(result)
    urls = [url.split('&')[0] for url in urls]
    return urls

In [4]:
keyword = "瓶子君 吐槽LoveLive星團第二季,1"
urls = YouTube_Search_Tool(keyword)
print(urls)

['https://www.youtube.com/watch?v=rsK89IZBdbs']


### 2. 資料前處理
#### (1) 下載音檔
請注意，自從2024/07起，`PyTube`就出現了`HTTPError: HTTP Error 400: Bad Request`的問題。因此這裡先手動下載影片

In [5]:
def url_download(urls):
    yt = YouTube(urls[0])
    print(yt.title,yt.watch_url)
    stream = yt.streams.filter(only_audio=True).first()
    stream.download(filename='./Ch9/Liella_comment.m4a')

In [6]:
# url_download(urls)

#### (2) 音檔轉文字
利用我們的老朋友Whisper建立文字檔案

In [7]:
client = OpenAI()

In [8]:
def audio_to_text(file_path):
    audio_file = open(file_path, "rb")
    transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file)
    return transcript.text

In [9]:
text=audio_to_text('./Ch9/Liella_comment.m4a')
print(text)

是啊 我也没有想到在还有整整一个季度的新作没有播的时候 年度最逆天动画的比赛就已经被他杀死 而捧起冠军的就是这部 LOVE LIVE SUPERSTAR第二季 是的 连最终回头没播就已经失去悬念了 我知道很多人对于LOVE LIVE星团的理解还挺流哉 太好听了吧 简直就是天籁 诚然唐可可这个别具一格的社交恐怖分子确实给啦啦引了波大流 而我也恰好是其中的一员 星团前三季的水准确实堪称历代从未如此美妙的开局 确实够格成为年轻人第一部LOVE LIVE 只可惜在后半段就迅速滑落为一般偶像样板戏 音乐链的故事更可称烂俗无比 一整季忙活下来不仅角色没塑造明白 还只拿了个外卡冠军 纯纯的高开低走 但众所周知LOVE LIVE系列人均两季打底 所以不要紧 这一切的问题都会在第二季被解决的吧 被解决的吧 解决的吧 觉得吧 的吧 啊吧 并没有 策划家四人了 策划家四人这个消息一出很多人纷纷狂喜 啊死的干净吗 还有剩的吗 后得知不是 是真的加了四个人 听得骂声一片 日比骂推前 吃香太难看了 但是当时想到会中两集 没想到会中到这种程度 那简直是FE中的拉提菲 LPL中的潘大C 是游戏业界的孔纳米 假面骑士界的灵依 是立绘界的水晨 是动物界的丁真 大概就是这种程度 编剧不能说是开摆了 只能说是开席了 这剧情写的让程序员山库跑路 有一种蓄意报复的美 唯一纯路人拳头都摁了 不敢想粉丝看完的心理活动 那我就来实际掰扯掰扯 第二季都发生了什么 事先声明 如果你觉得完全无法相信以下内容 恭喜你 你是正常人 在第一季结尾的拉拉决赛 萨尼帕捧起冠军奖杯 却难掩颓势 星团虽刀零比三 仍尽显王者风范 只有星团的五个人升到了二年级 本来按照原计划 达到决赛的班底 氛围和化学反应都有 就缺一手魔盒 亚军班底再来一年呗 届时三年级的萨尼帕 也是年老色衰 拙日黄花 而一年级的小BK 肯定也是乳臭未干 不成气候 但是我们的大香老师 突然又开始想点子了 众所周知 星团第二季又名大香英雄传 又名色谷香音的忧郁 香音想要什么就得有什么 神说要有光 大香说不行 于是就不能有光 凡是大香老师对的 那他太英明了 如果他错了 参见上一天 而大香老师 疑似是被路上偶遇的 土妹子鹰小鹿 叫前辈叫嗨 突然提出 我们应该广招一年级新人 即使那样会削弱团队的集战力 没事 冠军不是唯一的目的 重要的是让大家明白 做school idol很快乐

### 3. 使用 RAG 處理資料
利用上一章的`RecursiveCharacterTextSplitter`做切割

In [10]:
text_splitter = RecursiveCharacterTextSplitter(separators=[' '], chunk_size=300, chunk_overlap=20)
splits = text_splitter.split_text(text)

In [11]:
pprint(splits)

[
    '是啊 我也没有想到在还有整整一个季度的新作没有播的时候 年度最逆天动画的比赛就已经被他杀死 而捧起冠军的就是这部 
LOVE LIVE SUPERSTAR第二季 是的 连最终回头没播就已经失去悬念了 我知道很多人对于LOVE LIVE星团的理解还挺流哉 
太好听了吧 简直就是天籁 诚然唐可可这个别具一格的社交恐怖分子确实给啦啦引了波大流 而我也恰好是其中的一员 
星团前三季的水准确实堪称历代从未如此美妙的开局 确实够格成为年轻人第一部LOVE LIVE 
只可惜在后半段就迅速滑落为一般偶像样板戏 音乐链的故事更可称烂俗无比 一整季忙活下来不仅角色没塑造明白 
还只拿了个外卡冠军 纯纯的高开低走',
    '还只拿了个外卡冠军 纯纯的高开低走 但众所周知LOVE LIVE系列人均两季打底 所以不要紧 
这一切的问题都会在第二季被解决的吧 被解决的吧 解决的吧 觉得吧 的吧 啊吧 并没有 策划家四人了 
策划家四人这个消息一出很多人纷纷狂喜 啊死的干净吗 还有剩的吗 后得知不是 是真的加了四个人 听得骂声一片 日比骂推前 
吃香太难看了 但是当时想到会中两集 没想到会中到这种程度 那简直是FE中的拉提菲 LPL中的潘大C 是游戏业界的孔纳米 
假面骑士界的灵依 是立绘界的水晨 是动物界的丁真 大概就是这种程度 编剧不能说是开摆了 只能说是开席了 
这剧情写的让程序员山库跑路 有一种蓄意报复的美',
    '有一种蓄意报复的美 唯一纯路人拳头都摁了 不敢想粉丝看完的心理活动 那我就来实际掰扯掰扯 第二季都发生了什么 
事先声明 如果你觉得完全无法相信以下内容 恭喜你 你是正常人 在第一季结尾的拉拉决赛 萨尼帕捧起冠军奖杯 却难掩颓势 
星团虽刀零比三 仍尽显王者风范 只有星团的五个人升到了二年级 本来按照原计划 达到决赛的班底 氛围和化学反应都有 
就缺一手魔盒 亚军班底再来一年呗 届时三年级的萨尼帕 也是年老色衰 拙日黄花 而一年级的小BK 肯定也是乳臭未干 不成气候 
但是我们的大香老师 突然又开始想点子了 众所周知 星团第二季又名大香英雄传 又名色谷香音的忧郁 香音想要什么就得有什么 
神说要有光',
    '香音想要什么就得有什么 神说要有光 大香说不行 于是就不能有光 凡是大香老师对的 那他太英明了 如果他错了 
参见上一天 而大香老师 疑似是被路上偶遇的 土妹子鹰小鹿 叫前辈叫嗨 突然提出 我们应该广招一年级新人 
即使那样会削弱团队的集战力 没事 冠军不是唯一的目的 重要的是让大家明白 做school idol很快乐 懂了 就是玩happy game 
完全对校园偶像没概念 没事 看个视频就行了 跟不上我们的训练强度 没事 开百就行了 调整公园老头老太太强度 看到录像里 
去年我们输了决赛 如丧考笔的表情了吗 也太快乐了吧 快加入我们和我们一起坐牢吧 到最后还得是新人倡议 把训练强度加回来',
    '到最后还得是新人倡议 把训练强度加回来 实在是安崩了不 但是下一集里大翔老师又说 必须要获胜才能让大家开心 
所以你也知道 你们本意要争取胜利 但就是不选能赢的玩法 偏选弹幕最多的玩法 疑似是怕你挨晚期了 于是加入新人的第一个考验
偶像纪 列拉一号种子参赛排面拉满 还有拉拉未免冠军 萨尼帕官方背书称之为头号劲敌 顶着最大的光环 和最高的流量进入比赛 
那必然是轻轻松松的 被打成世界第一吹吉巴 蝉联安慰奖 被一个初三的小姚贝娜 直接畅攻硬实力打爆 
连聋哑人都能感受到的视听差距 好吧这说法夸张了 其实感受不到那么大的差距 因为列拉的演出动画里直接跳过了 不播了 
直接连对比都别比了 镜头直接切到5张',
    '直接连对比都别比了 镜头直接切到5张 垮几个小猫劈脸的表情 大翔老师还被对面贴脸嘲讽 更是直接开启飞行模式 
看不见听不见 嘻嘻哈哈爸爸开始自立 我让他隔天进来 第一句话翻一下就是 我们吹的响比赛拉 忙活一年多毛都没拿 
就我们这样的说要夺冠真的配吗 啊牛逼 这是队长能说出来的话 本来指望他来一波拧绳 结果他就差直接给姐妹们吊上了 
还好3D署姐妹及时的制止了他 说其实你们已经很强了 看看我们学校的奖杯贵吧 要不是千神和其他的天图 
我们就只能摆个花盆上去了 那为了回应学校师生的厚爱 是时候痛定思痛努力 六人组合训练了吧 不我们应该招收更多新人 
进一步稀释纯度吧 我们烂不要紧',
    '进一步稀释纯度吧 我们烂不要紧 反正这个学校有的是比我们更烂的 于是大乡老师又开始四处留情 坑蒙拐骗 
终于勾到这么一对森林冰火人 一个节秋爱迪生 天天导致各种发明 要不是当学院偶像耽误了 他非得提前一博年给哆啦A梦整出来 
一个是咧啦超话主持人 纯纯的追星小妹 天天拿个望远镜 一边留哈喇子 一边当侦察兵偷看咧啦训练 一边发微博 
这俩人还能上演一波破产版的例子与青鸟 眼中点中点之 互相喜欢但互相不理解 我是万万没想到 因为这俩都是好朋友 
小红喜欢学院偶像 但为了对方忍痛和他一起成立了科学部 而小蓝知道对方喜欢学院偶像 所以希望他去加入咧啦 
因此不惜以身犯险 率先加入咧啦做激将法 小红一看',
    '率先加入咧啦做激将法 小红一看 卧槽我这绑一大哥 当这么久都只敢say你 直接就加入你就打我的脸是吧 
明明俩人都是互相在乎对方挺好的事 结果差点快进到扯头发 又是大乡老师的一顿神奇嘴炮 及时安抚住小红 哎呀他可太会嘴炮了 
建议改名叫大炮老师吧 毕竟名字就叫canon嘛 说啊小蓝喜欢你 所以他肯定也喜欢学院偶像 什么爱屋及乌啊 
照着逻辑那些喜欢西海岸匪帮说唱的人 还喜欢什么我都不敢想了 就这样森林兵伙双双加入 这下终于可以狠狠的开始特训了吧 
哈哈当然不行 加四人一个都不能少 隔天这个叫CEO的夏美跑来掐痰业务 说我可以帮你们营销推广 所以统统要归他所有 
还得是大乡老师 当场不看内容直接签合同',
    '还得是大乡老师 当场不看内容直接签合同 然后他们就开始拍生活区视频 开始不要笑挑战 开始蹭饭挑战 
参加男生女生向前冲 开始对对子 开拍抖音热门 训练的时候都要被相机怼脸 终于回味未来的锦终于查出猫腻 
你这家伙不会就是为了蹭我们一波赚米吧 谁知道旁边自己的老队友先出来怼自己 真的蹦不住了 这第二季唐可可和锦的关系 
直接从欢喜冤家变成山中北斗 可可能半天不说话 一张嘴就是怼小锦 攻击性简直堪比日本天皇 
我都怀疑花田实惠是不是受了什么刺激 写cpc的口味变重了 其实锦不是被动遭受霸凌 只是忘记了安全词 花田啊 
不会写互动可以不写 别恶心人了 结果小蓝也得出同样的结论 截问CEO',
    '结果小蓝也得出同样的结论 截问CEO CEO说我确实赚大米了 但是你们签了合同的 众人大怒 啊哪个傻逼做主签的 
哦哦哦签的好啊 拍视频好开心啊 哈哈哈哈 大乡老师先锋勇巧 授予天启 新的一集新的绝活 一年级三人组说 暑假要分开训练 
说被咧了能在啦啦夺冠尽一份力 我们什么都会做的 我心说那太好了 你们马上退 严筹几个月就要地区预选 还不合連 
到时候别人台上唱歌跳舞 你们演东白网似的 签人稍微要求提高强度 就开始愁眉苦脸 还得让CEO这个前沿里买房的谢BRO 
给他们一通训话才好 啊有面 你们懂有面吗 有面 有面面 面面面面面 话说CEO也不是什么坏人 他只是没逼 他确实懂有面 
而且还有过很多',
    '他确实懂有面 而且还有过很多 以前的梦想当科学家 但学习堪比大雄 后来梦想拿奥运金牌 但运动堪比大雄 再后来想当模特
但身高堪比大雄 那啥 要不你去试试射击 后来终于明白梦想都是泡沫 一心搞钱了 拍摄期间全程跟队一起训练 天赋异禀 
只需要个台阶下 自然就可以重新拥有有面 还是交给始终苦口婆心 试图劝下美从良的土妹子 英小露奈 现在九人里了 
终于集结完毕 而其中的某些人 直到现在 我也不知道为啥要加入 用一段总结视频 总结这几个新人就是 我喜欢钱 我恨你们所有人
那边前方一年级吃紧 后面二年级紧持 练练这一季基本捧人 台词全是 恩啊这是 哎嘿 游河 去你的吧 序谋依旧也要搞个大伙 
从某天起',
    '序谋依旧也要搞个大伙 从某天起 练练的眼圈就黑的像L一样 不知道的还以为他和卡蜜搏斗了 结果只是单纯网瘾发作 
干完手游玩主机 玩完主机电竞梦 荒野行动扛大狙 死不拉痛秀八五 瓦罗兰特奥丁狗 三枪拍案惊奇 满脑子划斩跳 
别参加什么G8拉拉了 直接加入大发明 拯救日本电竞吧 直到终于瞒不住 不得不坦白的时候 大家哈哈大笑 就这啊 然后跟他一起玩
哇 太好玩了吧 这游戏好好玩啊 哈哈 哈哈 哈哈 我只能说这世界笑了 于是我合群的一起笑了 花田 你太发霉油麦了 
而且能解释一下 为啥玩NS会谈steam的成就吗 下一集 里拉相对不乱不惊的拿下了地区预选 以他们的训练方式能跳这么起 
只能说',
    '以他们的训练方式能跳这么起 只能说 小蓝终于发明出了 环太平洋中的生物同步技术 而且这舞台抽象的跟台观一样 
不知道的还以为是给安倍复活赛的赛前电场演出呢 几家欢喜几家愁 里拉在庆祝进入了地区决赛 未免冠军撒尼帕居然抱了个大拢 
输给了此前的初三小哥王 意思是新人拉满不坑当C 突发网瘾的晋级了 兢兢业业的未免冠军冤死于堂 拉不赖五 
不愧是无法预测的命运之舞台啊 虽然赢了 但居安思危 千妈仔细研究demo后 发现一年级四个人 多少还是有点拉了 
这地区决赛不包赢啊 一年级一看 没关系 我们什么都会做的 比如呢 把对手的疯狂星际四换成华莱士 在微博大字报带对手节奏 
不是这真的是正面人物吗',
    '不是这真

這次我們要使用的Meta的`Faiss`作為向量搜尋的程式庫（不然一直算cosine similarity感覺很low）。此程式庫支援gpu。我們一樣將文字檔用`OpenAIEmbeddings`做編碼，接著存放到資料庫裡

In [12]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_texts(splits, embeddings)
db.save_local("./Ch9/youtube_db")

設定參數`allow_dangerous_deserialization=True`才能夠載入資料檔案

In [13]:
new_db = FAISS.load_local(folder_path="./Ch9/youtube_db", embeddings=embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search('對這部電影的感受')
pprint(docs[0])

Document(
    page_content='不是这真的是正面人物吗 听起来他们要对付的敌人叫白雪公主啊 还是紧惦念着可可不夺冠就要被迫回国 
直接扮演白脸 下一场 亚军员班人马上 一年级一听肯定不服 我们会努力的 我们什么都会做的 我们有我们的觉悟啊 
来大声说出你们的觉悟 啊 我 我X的 那你们前面八级班都在演牛魔大仇宾啊 真的纯纯的逆天 景也是心软了 再被赶来的可可一催 
两人直接抱头痛哭 这俩的关系直接当场倒退一年 崩城西的人设也是重新被反向夺社会了 但就这么感人的环境下 
大乡老师还惦记K头 还好前妈一个抢手猎卢给他屌了下来 不然封神榜要改叫耻辱路子 太棒了 现在终于三军用命一心向前了 
再经历了何处的锤炼 地区决赛终于上演'
)

### 4. 串起Chain
都用老方法即可

In [14]:
chat_model = ChatOpenAI()
str_parser = StrOutputParser()
template = ("""
    請根據以下內容加上自身判斷使用繁體中文回答問題:\n
    {context}\n
    問題: {question}
    """)
prompt = ChatPromptTemplate.from_template(template)
retriever=new_db.as_retriever()

In [15]:
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | chat_model | str_parser

In [16]:
query = "影評對這部動畫的感受為何?請詳細說明"
print(chain.invoke(query))

根據這些文件內容，影評對這部動畫的感受似乎非常負面。影評者提到劇情充滿回光返照，挣扎卻又被踹回去，角色塑造離譜，情節離譜，並且對主要角色和劇情發展進行了批評。影評者也表達了對劇情走向和角色命運的擔憂，並提到了對該動畫第二季的失望。整體來說，影評者對這部動畫持有非常批評和負面的態度，認為劇情和角色塑造都存在問題，導致觀眾無法接受和理解。


In [17]:
query = "總結影評中對於香音這個角色的描述？"
print(chain.invoke(query))

根據影評中的描述，香音這個角色被形容為一個有著蓄意報復的美感，有著獨特的個性和態度。她在劇中展現出對於成為school idol的熱情和堅持，同時也展現出對於團隊的貢獻和影響力。她的行為和決定常常讓其他角色感到驚訝和困惑，但同時也帶給整個團隊活力和挑戰。


### 5. 建立能持續對話的程式
以下的程式碼用以建立一個聊天機器人，以使用者的詢問作為input，並以模型的回答作為output。完整的前後端程式可參考這份專案[Replit 專案](https://replit.com/@flagtech/Youtubechatbot)

In [18]:
def search_url(query):
    result = tool.invoke(query + ",1")
    urls = eval(result)
    urls = [url.split('&')[0] for url in urls]
    return urls

In [19]:
def rag(text):
    splits = text_splitter.split_text(text)
    db = FAISS.from_texts(splits, embeddings)
    db.save_local("/content/drive/MyDrive/youtube_db")
    new_db = FAISS.load_local(folder_path="/content/drive/MyDrive/youtube_db",
                              embeddings=embeddings,
                              allow_dangerous_deserialization=True)
    retriever=new_db.as_retriever()
    chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | chat_model | str_parser
    
    return chain

In [20]:
while True:
    msg = input("您要查詢的影片關鍵字是？：")
    if not msg.strip():
        break
    urls = search_url(msg)
    url_download(urls)
    print('處理中...')
    text=audio_to_text('/content/audio.mp3')
    print('處理完畢, 可開始問答')
    chain = rag(text)
    while True:
        msg = input("我說:")
        if not msg.strip():
            break
        response = chain.invoke(msg)
        print(response)

您要查詢的影片關鍵字是？： 


## 9-2 辦公室常見檔案問答機器人
將常見的Word，Excel，PowerPoint檔案匯入，並建立公司內部的聊天機器人。這些檔案的匯入可以用`UnstructuredFileLoader`來實現。包含前後端的完整專案可參考[Replit 專案](https://replit.com/@flagtech/Officefilebot)以及[GitHub 可讀圖片 PDF 的 Source code](https://github.com/FlagTech/F4763)

In [21]:
def office_file(file_path):
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load()
    return docs

### 1. PowerPoint文件的處理
先使用指令`pip install python-magic-bin==0.4.14`下載需要的包

In [22]:
ppt_docs = office_file('./Ch9/example_ppt.pptx')

In [23]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n"], chunk_size=150, chunk_overlap=0)
splits = text_splitter.split_documents(ppt_docs)
for i in splits[:2]:
    print(i.page_content)
    print('_'*10)

立法院第10屆第3會期經濟委員會

國家發展委員會業務報告

國家發展委員會

主任委員 龔明鑫

全球經濟恢復成長

國際機構全球經濟成長率預測

單位：%

2021年1月預測

2021年2月預測

2021年3月預測
__________
經濟風險仍高：疫情尚難緩和、美中衝突長期化、全球債務攀升金融風險升高、不均衡復甦

2021年展望

實質輸出成長+5.05%

全球貿易量回升

數位轉型商機

新興科技應用需求仍殷

2021年(f)  (+4.64%)

單位：%

近7年新高!!

實質民間消費成長+3.74%
__________


In [24]:
def rag(splits, name):
    db = FAISS.from_documents(splits, embeddings)
    db.save_local(folder_path="/content/drive/MyDrive/office_db", index_name=name)
    new_db = FAISS.load_local(folder_path="/content/drive/MyDrive/office_db", 
                              index_name=name,
                              embeddings=embeddings,
                              allow_dangerous_deserialization=True)
    retriever=new_db.as_retriever()
    chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | chat_model | str_parser
    
    return chain

In [25]:
ppt_chain = rag(splits, 'ppt')

In [26]:
print(ppt_chain.invoke("台灣的創新產業如何?"))

根據提供的資料，台灣的創新產業發展策略包括擴大投融資加速新創成長、形塑國家新創品牌強化商機拓展、深化物聯網國際夥伴關係、打造國際級亞矽創新聚落等。台灣已經在吸引國際大廠創新研發資源、推動智慧城鄉服務、活絡人才和法規調適等方面取得一定成就。根據世界經濟論壇的評比，台灣被評為全球4大超級創新國之一，並且物聯網已成為兆元產業。台灣也已通過多家企業的投資，帶動了相當大的資金流動。整體來說，台灣的創新產業發展走在一個積極的方向上，希望能成為全球經濟的關鍵力量。


### 2. Word文件的處理

In [27]:
word_docs = office_file('./Ch9/example_word.docx')

In [28]:
text_splitter = RecursiveCharacterTextSplitter(separators=["第[一二三四五六七八九十]+章"],
                                               is_separator_regex=True,
                                               chunk_size=200,
                                               chunk_overlap=10)
splits = text_splitter.split_documents(word_docs)
for i in splits[:2]:
    print(i.page_content)
    print('_'*10)

常見詐騙手法話術解析與預防策略
__________
第一章　假網拍詐騙

手法及話術解析：

民眾透過臉書、LINE或知名拍賣網站從事網路購物，詐騙集團便利用當前最新款的3C產品、限量球鞋、名牌包或熱門演唱會門票等，以明顯低於市價之價格誘引民眾下單並要求以LINE或Messenger私下交易。等被害人匯款後卻不出貨，且失去聯繫；或以貨到付款方式取貨開箱後，才發現是劣質商品。

預防策略

網購商品應慎選優良有信用之網路商家，透過面交方式或選擇提供第三方支付之網購平臺，以保障雙方權益並減少消費糾紛。

避免透過LINE、Messenger等通訊軟體與賣家私下聯繫交易，以免求助無門。

應加強查核網路商家的真實性，如粉絲專頁成立時間短、粉絲或追蹤人數過少等，可立即至經濟部網站查詢公司名稱、地址等基本資料。


__________


In [29]:
word_chain = rag(splits, 'word')

In [30]:
print(word_chain.invoke("如何防治詐騙電話?"))

根據提供的資訊，防治詐騙電話的方法包括：

1. 接獲陌生來電告知涉入刑案或收到疑為政府機關公文書時，應先小心求證，詢問對方單位、職稱、姓名等，掛斷電話後再向對方單位求證。

2. 警察機關不會在電話中做筆錄，檢警調等單位處理司法案件，會以通知書通知涉案人到案說明或至指定機關繳交款項。

3. 當接到以親友名義來電借錢時，務必以舊有電話向當事人親自確認，切勿貿然前往銀行匯款，以免遭詐騙。

4. 如果訊息中帶有不明連結，請先向發送訊息的朋友確認，避免點擊可能是詐騙的連結。

這些方法可以幫助人們在接到可疑的詐騙電話時，提高警覺並避免成為詐騙集團的受害者。


### 3. Excel

In [31]:
excel_docs = office_file('./Ch9/example_excel.xlsx')

In [32]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n\n"], chunk_size=150, chunk_overlap=0)
splits = text_splitter.split_documents(excel_docs)
for i in splits[:2]:
    print(i.page_content)
    print('_'*10)

項次
項目
詳細說明


1
營運時間
1.週二至過五09:00~17:00。2.寒暑假期間及連續假期(收假日除外)延長營運至20:00。3.週六、週日或收假日延長營運至18:00。4.週一(寒暑假期及連續假期除外)及農曆除夕為休園日不對外開放。5.營運時間如有調整將另行公告。
__________



2
門票資訊
全票：30元、優待票：15元、團體票：30人以上按票價打7折;◎全票：一般民眾◎優待票：1.年滿7歲而未滿12歲之兒童(出示身分證件);2.持有效學生證之在學學生(外國在學學生須持有效期之國際數位學生證);3.持有臺北市政府核發之低收入戶證明;◎免費入園：1.6歲(含)以下兒童;2.設籍臺北市之55歲(含)以上原住民;3.65歲(含)以上長者;4.臺北市退休公教人員持退休證者;5.身心障礙者及其必要之陪伴者1人;6.持志願服務榮譽卡之志工;7.持臺北市國小數位學生證者(私立學校為一般學生證);8.週六及連續假期(連續假期最後一天除外)16:00後入園;9.平常日凡30人(含)以上之團體，依現行團體預約方式，於前1日17:00前完成預約作業;10.設籍臺北市之市民，於週三入園;11.持有臺北市之義勇人員(含交通、警察、消防)及民防人員身分證明。
__________


In [33]:
excel_chain = rag(splits, 'excel')

In [34]:
print(excel_chain.invoke("我身高超過 120 公分, 我可玩那些遊樂設施?"))

根據提供的資料，您身高超過120公分的話，可以玩以下遊樂設施：
1. 水果摩天輪
2. 魔法星際飛車


### 4. CSV文件的處理
注意`create_pandas_dataframe_agent`的版本仍不穩定，因此運作起來可能會有危險。在此先全部comment掉。

In [35]:
df = pd.read_csv("./Ch9/example_csv.csv", skiprows=1)

In [36]:
# from langchain_experimental.agents import create_pandas_dataframe_agent

# agent = create_pandas_dataframe_agent(llm=chat_model,
#                                       df=df,
#                                       prefix='回答請使用繁體中文',
#                                       agent_type="openai-tools",
#                                       verbose=True)

In [37]:
# result = agent.invoke({"3月收盤價的平均值是?"})
# print(result['output'])
# print(df['收盤價'].mean())